In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
df = pd.read_csv('./App_Behaviour/Subcription App Behaviour.csv')
df

,user,first_open,dayofweek,hour,age,screen_list,numscreens,minigame,used_premium_feature,enrolled,enrolled_date,liked
0,235136,14:51.3,3,02:00:00,23,"idscreen,joinscreen,Cycle,product_review,ScanP...",15,0,0,0,NaN,0
1,333588,16:00.9,6,01:00:00,24,"joinscreen,product_review,product_review2,Scan...",13,0,0,0,NaN,0
2,254414,19:09.2,1,19:00:00,23,"Splash,Cycle,Loan",3,0,1,0,NaN,1
3,234192,08:46.4,4,16:00:00,28,"product_review,Home,product_review,Loan3,Finan...",40,0,0,1,11:49.5,0
4,51549,50:48.7,1,18:00:00,31,"idscreen,joinscreen,Cycle,Credit3Container,Sca...",32,0,0,1,56:37.8,1
5,56480,58:15.8,2,09:00:00,20,"idscreen,Cycle,Home,ScanPreview,VerifyPhone,Ve...",14,0,0,1,59:03.3,0
6,144649,33:18.5,1,02:00:00,35,"product_review,product_review2,ScanPreview",3,0,0,0,NaN,0
7,249366,07:49.9,1,03:00:00,26,"Splash,Cycle,Home,Credit3Container,Credit3Dash...",41,0,1,0,NaN,0
8,372004,22:01.6,2,14:00:00,29,"product_review,product_review2,ScanPreview,Ver...",33,1,1,1,24:54.5,0
9,338013,22:16.0,4,18:00:00,26,"Home,Loan2,product_review,product_review,produ...",19,0,0,1,31:58.9,0


In [3]:
df['enrolled_date'].isna().sum()

18926

In [4]:
df.isna().sum()

user                        0
first_open                  0
dayofweek                   0
hour                        0
age                         0
screen_list                 0
numscreens                  0
minigame                    0
used_premium_feature        0
enrolled                    0
enrolled_date           18926
liked                       0
dtype: int64

In [5]:
#HyperParameters
n_trees = 100

In [6]:
# screen_vocab = list(set((','.join(df['screen_list'])).split(',')))
# screen_index = {s:i for i,s in enumerate(screen_vocab + ['<start>', '<end>'])}
# vocab_size = len(screen_index)
# ind_screen_indices = np.array(df['screen_list'].apply(lambda x : [screen_index[w] for w in ['<start>'] + x.split(',') + ['<end>']]))
# flat_indices = np.concatenate(ind_screen_indices)
# cond_freq_mat = np.zeros((vocab_size-1, vocab_size))
# un_freqs,freqs = np.unique(flat_indices, return_counts = True) # un is in ascending order
# flat_indices = None
# pairs_indices = [[(x[i], x[i+1]) for i in range(len(x)-1)] for x in ind_screen_indices]
# flat_pairs_idx = np.concatenate(pairs_indices)
# unique_pairs,pair_freqs = np.unique(flat_pairs_idx, return_counts = True, axis = 0)
# flat_pairs_idx = None
# cond_freq_mat[(tuple(unique_pairs[:, 0]), tuple(unique_pairs[:, 1]))] = pair_freqs
# cond_prob = cond_freq_mat/freqs[:-1].reshape(vocab_size-1, 1)
# pair_freqs = None
# unique_pairs = None

In [7]:
screen_vocab = list(set((','.join(df['screen_list'])).split(',')))
screen_index = {s:i for i,s in enumerate(screen_vocab)}
max_len = np.max(df['numscreens'].values)
total_words = df['numscreens'].sum()
vocab_size = len(screen_index)
ind_screen_indices = np.array(df['screen_list'].apply(lambda x : [screen_index[w] for w in x.split(',')]))
flat_indices = np.concatenate(ind_screen_indices)
padded_indices = np.array([x + [vocab_size]*(max_len - len(x)) for x in ind_screen_indices])
ind_screen_indices = None
un_freqs,freqs = np.unique(flat_indices, return_counts = True) # un is in ascending order
flat_indices = None

print(max_len)
k = 3
num_cols = padded_indices.shape[1]
tiled_indices = np.zeros((2, padded_indices.shape[0],  k*(num_cols-k)))
for i in range(k):
    tiled_indices[1, :, i:k*(num_cols-k-1)+i+1:k] = padded_indices[:, i+1:num_cols-k+i+1]
tiled_indices[0, :, :] = np.repeat(padded_indices[:, :-k], k, axis = 1)
padded_indices = None
print(tiled_indices.shape)
tiled_indices = tiled_indices.reshape(2, tiled_indices.shape[2]*tiled_indices.shape[1]).T
print('reshape_done')
unique_pairs,pair_freqs = np.unique(tiled_indices, return_counts = True, axis = 0)
print(unique_pairs)
# tiled_indices = None
# cond_freq_mat = np.zeros((vocab_size+1, vocab_size+1))
# cond_freq_mat[(tuple(unique_pairs[:, 0]), tuple(unique_pairs[:, 1]))] = pair_freqs
# neigh_prob = (cond_freq_mat + cond_freq_mat.T)/freqs.reshape(vocab_size, 1)
# print(np.max(neigh_prob, axis = 1))


325
(2, 50000, 966)
reshape_done
[[  0.   1.]
 [  0.   4.]
 [  0.   5.]
 ...
 [136. 135.]
 [136. 137.]
 [137. 137.]]


In [8]:
unique_pairs = unique_pairs.astype('int32')
cond_freq_mat = np.zeros((vocab_size+1, vocab_size+1))
cond_freq_mat[unique_pairs[:, 0], unique_pairs[:, 1]] = pair_freqs
neigh_prob = (cond_freq_mat[:-1, :-1] + cond_freq_mat[:-1, :-1].T)/(k*freqs.reshape(vocab_size, 1))
# neigh_prob = neigh_prob - 1
print(np.max(neigh_prob, axis = 1))
# neigh_prob[neigh_prob == 0] = -1
# neigh_prob = neigh_prob*(1 - np.eye(vocab_size))


[0.30757518 0.29674607 0.30755064 0.32051282 0.28255141 0.29490784
 0.30274262 0.55242227 0.3092006  0.22281312 0.21701614 0.28020184
 0.23469388 0.26315789 0.33333333 0.1567914  0.28100471 0.21232683
 0.26485888 0.30816479 0.22934531 0.20833333 0.13930348 0.22383952
 0.27180784 0.29166667 0.28962857 0.21226911 0.23875969 0.22330097
 0.25943647 0.23892386 0.19256869 0.1159622  0.30984508 0.10226296
 0.29676512 0.25783476 0.23630875 0.21232123 0.25819872 0.15816571
 0.20227082 0.20643594 0.18918919 0.29569892 0.248157   0.2851711
 0.33333333 0.33333333 0.3060686  0.70563742 0.34161373 0.21027206
 0.29487179 0.280973   0.26243967 0.26666667 0.17772318 0.25
 0.19619048 0.44725326 0.33333333 0.12507937 0.2165319  0.19012346
 0.32282445 0.24538197 0.29508197 0.33990295 0.28205128 0.28879892
 0.27777778 0.30552764 0.26888519 0.28701953 0.29796805 0.78097983
 0.33333333 0.21670481 0.33333333 0.14552846 1.3089934  0.6371308
 0.21661055 0.20634921 0.16980287 0.2        0.21391253 0.22222222
 0.

In [9]:
# #initialize  word vectors
# wvec_dim = vocab_size
# word_vectors = np.identity(vocab_size)
# # for i in range(wvec_dim):
# #     word_vectors[:, i] = np.random.uniform(size = vocab_size)

# #learning rate and num_iterations
# lr = 5e-3
# prev_sim = -1e5
# num_iterations = 1000
# similarity = 1e6
# ep = 0
# epsilon = 1e-10
# while ((ep < num_iterations)):
#     ep += 1
#     orig_vectors = np.array(word_vectors)
#     for v in range(vocab_size):
#         word_vectors[v, :] += lr*np.sum((orig_vectors - orig_vectors[v, :])*((orig_prob[v, :]-0.001).reshape(vocab_size, 1)), axis = 0)
#         word_vectors[v, :] /= np.linalg.norm(word_vectors[v, :])
# #     loss = np.sum((np.sum((np.tile(word_vectors, (vocab_size, 1, 1)) - word_vectors.reshape(vocab_size, 1, wvec_dim))**2, axis = 2).squeeze())*orig_prob, axis = (0,1))
#     similarity = np.sum(word_vectors@(word_vectors.T)*orig_prob, axis = (0,1))
#     if(similarity  < prev_sim):
#         print("Overshoot")
#         lr *= 0.5
#     prev_sim = similarity
#     print(similarity)
    
# print(word_vectors)

In [10]:
wvec_dim = vocab_size
word_vectors = np.zeros((vocab_size, vocab_size))

word_vectors[0, 0] = 1

for v in range(1, vocab_size):
    dot = 0
    for d in range(v):
        if (word_vectors[d,d] != 0):
            word_vectors[v, d] =  (neigh_prob[v, d] - dot)/(word_vectors[d,d])
        dot = np.dot(word_vectors[v, :d+1], word_vectors[d+1, :d+1])
    if(np.sum(word_vectors[v, :v]**2) < 1):
        word_vectors[v, v] = np.sqrt(1 - np.sum(word_vectors[v, :v]**2))
    else:
       word_vectors[v, :v] /= np.linalg.norm(word_vectors[v, :v])


In [11]:
idf = np.zeros(vocab_size)
sentence_vec = np.zeros((df.shape[0], wvec_dim))

ind_screen_indices = np.array(df['screen_list'].apply(lambda x : [screen_index[w] for w in x.split(',')]))

for x in ind_screen_indices:
    idf[list(set(x))] += 1

for i in range(df.shape[0]):
    x = tuple(ind_screen_indices[i])
    sentence_vec[i, : ]= np.sum(word_vectors[x, :]*idf[list(x)].reshape(len(x), 1), axis = 0)

df_freq = pd.DataFrame(sentence_vec, columns = ['V' + str(i) for i in range(wvec_dim)])

In [12]:
# median_len = df['numscreens'].median() # 18
# numfeatures = 25
# sentence_probs = []
# for x in pairs_indices:
#     xn = np.array(x)
#     sentence_probs.append(cond_prob[(tuple(xn[:, 0]), tuple(xn[:, 1]))])
# pairs_indices = None

# new_screen_features = np.zeros((df.shape[0], numfeatures))
# for m, x in enumerate(sentence_probs):
#     sum = 0
#     for i in range(min(len(x), numfeatures)):
#         sum += np.log(x[i])
#         new_screen_features[m, i] = sum
# df_new_screen = pd.DataFrame(new_screen_features, columns = ['C' + str(i) for i in range(numfeatures)])

In [13]:
# freq_list = np.zeros((df.shape[0], vocab_size))
# for i in range(df.shape[0]):
#     for ind in ind_screen_indices[i]:
#         freq_list[i, ind] += 1
# freq_list = freq_list/vocab_size
# df_freq = pd.DataFrame(freq_list, columns = ['V' + str(i) for i in range(vocab_size)])

In [14]:
from sklearn.preprocessing import StandardScaler

first_open = df['first_open'].apply(lambda x : float(x.split(':')[0])*24*7 + float(x.split(':')[1]))
access_time = pd.Series(df['dayofweek']*24 + (df['hour'].apply(lambda x : float(x[:2]))), name = 'access_time')
first_open = first_open.to_frame()
access_time = access_time.to_frame()

df.drop(labels = ['screen_list', 'user', 'enrolled_date', 'first_open', 'dayofweek', 'hour'], axis = 1, inplace = True)
padded_indices = None
mean_screen = None
ind_screen_indices = None
scaler = StandardScaler()
df = pd.concat([first_open, access_time, df_freq, df], axis = 1)
col_list = [x for x in list(df.columns) if x  != 'enrolled']
scaler = StandardScaler()
df[col_list] = scaler.fit_transform(df[col_list])
df

,first_open,access_time,V0,V1,V2,V3,V4,V5,V6,V7,...,V133,V134,V135,V136,age,numscreens,minigame,used_premium_feature,enrolled,liked
0,-0.867172,-0.032562,-0.175438,-0.145569,8.974990,0.470096,-0.294384,-0.156167,-0.381871,-0.749688,...,0.025678,0.0,0.0,0.0,-0.807572,-0.387567,-0.347635,-0.455805,0,-0.444528
1,-0.770194,1.443310,-0.206314,-0.072709,-0.397275,0.975934,-0.129838,-0.094415,0.565066,-0.756674,...,0.025678,0.0,0.0,0.0,-0.715007,-0.514723,-0.347635,-0.455805,0,-0.444528
2,-0.596238,-0.995979,-0.474093,-0.651698,-0.123536,-1.724049,-1.161090,-0.359571,-1.358526,-0.973084,...,0.025678,0.0,0.0,0.0,-0.807572,-1.150505,-0.347635,2.193918,0,2.249579
3,-1.211111,0.479893,-0.383860,-0.151639,-0.358534,0.046266,1.954225,-0.160796,-0.214276,0.175256,...,-2.904956,0.0,0.0,0.0,-0.344746,1.201889,-0.347635,-0.455805,1,-0.444528
4,1.185595,-0.995979,0.086760,-0.049961,-0.027558,0.649069,1.666927,-0.132150,0.267464,0.377673,...,0.025678,0.0,0.0,0.0,-0.067050,0.693263,-0.347635,-0.455805,1,2.249579
5,1.630791,-0.524520,-0.355867,-0.299691,-0.219017,0.461945,1.559156,-0.208010,0.044831,-0.205584,...,0.025678,0.0,0.0,0.0,-1.085267,-0.451145,-0.347635,-0.455805,1,-0.444528
6,0.205561,-1.016477,-0.490525,-0.580941,-0.688069,-1.412625,-1.047963,-0.306693,-0.420854,-0.592493,...,0.025678,0.0,0.0,0.0,0.303210,-1.150505,-0.347635,-0.455805,0,-0.444528
7,-1.266968,-1.016477,-0.087917,-0.111881,0.444598,0.461945,-0.249486,-0.130315,0.304033,0.611712,...,0.025678,0.0,0.0,0.0,-0.529876,1.265467,-0.347635,2.193918,0,-0.444528
8,-0.427682,-0.504021,-0.010238,0.220864,0.591634,0.923008,0.592412,0.045351,1.477129,4.095261,...,0.025678,0.0,0.0,0.0,-0.252181,0.756842,2.876581,2.193918,1,-0.444528
9,-0.422792,0.479893,-0.096368,0.086394,-0.014057,1.131717,0.444599,-0.045544,1.059723,1.475410,...,0.025678,0.0,0.0,0.0,-0.529876,-0.133254,-0.347635,-0.455805,1,-0.444528


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

labels = df['enrolled']
features = df.drop('enrolled', axis = 1)
validation_acc = []
train_acc = []
X_Train, X_Test, Y_Train, Y_Test = train_test_split(features, labels, test_size=0.33, random_state=20)
# for depth in range(10, 100, 9):
#     model = RandomForestClassifier(n_estimators = n_trees, max_depth = depth)
#     model.fit(X_Train, Y_Train)
#     Y_Test_pred = model.predict(X_Test)
#     Y_Train_pred = model.predict(X_Train)
#     validation_acc.append(accuracy_score(Y_Test, Y_Test_pred))
#     train_acc.append(accuracy_score(Y_Train, Y_Train_pred))
model = RandomForestClassifier(n_estimators = 100, max_depth = 50)
model.fit(X_Train, Y_Train)
Y_pred = model.predict(X_Test)
acc = accuracy_score(Y_Test, Y_pred)
conf_mat = confusion_matrix(Y_Test, Y_pred)
print(acc)
print(conf_mat)
# plt.plot(range(10, 100, 9), validation_acc)
# plt.plot(range(10, 100, 9), train_acc)
# plt.show()
# print(max(validation_acc))

0.836
[[5335  910]
 [1796 8459]]


In [16]:
import xgboost as xgb

X_Train, X_Test, Y_Train, Y_Test = train_test_split(features, labels, test_size=0.1, random_state=20)
D_train = xgb.DMatrix(X_Train, label=Y_Train)
D_test = xgb.DMatrix(X_Test, label=Y_Test)

param = {
    'eta': 0.5, 
    'max_depth': 15,  
    'objective': 'multi:softprob',  
    'num_class': 2} 

steps = 20 

model = xgb.train(param, D_train, steps)

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Accuracy = {}".format(accuracy_score(Y_Test, best_preds)))

Accuracy = 0.8404
